# `swift_too` module

## Swift_PlanQuery example - querying planned *Swift* observations

### API version = 1.2

#### Author: Jamie A. Kennea (Penn State)

The Swift_PlanQuery class allows for querying the database of observations have have been planned for observation by Swift, otherwise known as the "Pre-Planned Science Timeline" (PPST). This allows the user to find out not only what was planned, but what will be observed by Swift in the future.

Two important caveats:

1. For past observations, what should have been down is mostly an academic exercise, as due to TOO uploads and GRBs, Swift's pre-planned observations can be obliterated. If you want to know what Swift *did* observe, do a Swift_ObsQuery

2. Swift's planning is only done days into the future, so if you want to know what is going to happen weeks from now, you're out of luck.

This therefore is mostly useful if you want to know what Swift is observing *today* and *tomorrow*.

In [1]:
from swifttools.swift_too import Swift_PlanQuery
from datetime import datetime,timedelta

### Setting up our username and secret key

Set up the class, giving our username and shared secret. These can be set up [on the Swift TOO website](https://www.swift.psu.edu/toop/too.php). After you log in, you will find your shared secret, and can modify it as necessary under the [Update Account Info](https://www.swift.psu.edu/toop/account_info.php) link. 

In [2]:
username = 'myuser'
shared_secret = 'mysharedsecret'

### First Query

First query example: What is Swift planned to observe now? Quite easy to construct. Here it is:

In [3]:
plan = Swift_PlanQuery(username,shared_secret,begin=datetime.utcnow())
plan

Begin,End,Name,Obs Number,Exposure (s)
2021-07-06 18:34:00,2021-07-06 19:04:00,ZTF21abdmevk,00014349008,1800


So the above should have reported back a single target. When you just give `start` with no `end`, it simply returns the observation which is taking place at the time given. So what is happening at this time tomorrow?

In [4]:
plan = Swift_PlanQuery(username,shared_secret,begin=datetime.utcnow()+timedelta(days=1))
plan

Begin,End,Name,Obs Number,Exposure (s)
2021-07-07 18:57:00,2021-07-07 19:22:00,XMMU J001528.9-391319,00032619042,1500


Did the above query return anything? If not, then there simply isn't a plan for tomorrow. So how can we determine how far into the future Swift has a plan for? The `ppstmax` parameter. Let's check it out:

In [5]:
f"PPSTs available up to {plan.ppstmax}"

'PPSTs available up to 2021-07-08 00:00:00'

This gives the date of the last planned target. If the query for what Swift is observing this time tomorrow didn't return anything, this date is probably the end of the UT day today. If the time you're running this is before midday on a Monday-Thursday, this is as expected. Typically the plan for tomorrow is submitted for upload around 12pm Eastern time. The exception to this rule is for the weekend plan. Typically on Thursday at 12pm we submit a plan for Friday and Saturday, and on Friday at 12pm we submit a plan for Sunday-Monday. This allows us to cover the weekend, while working a 5 day week.

### Second query example

Seeing a whole day's PPST. So let's see the whole day's Plan that goes farthest into the future.

In [6]:
dayplan = Swift_PlanQuery(username,shared_secret,begin=plan.ppstmax-timedelta(days=1),end=plan.ppstmax)
dayplan

Begin,End,Name,Obs Number,Exposure (s)
2021-07-06 23:40:00,2021-07-07 00:00:00,SGR 1830-0645,00013840016,1200
2021-07-07 00:00:00,2021-07-07 00:08:00,M31N 1923-12c,00014412005,480
2021-07-07 00:08:00,2021-07-07 00:18:00,SC1825,00013984048,600
2021-07-07 00:18:00,2021-07-07 00:26:00,OGP_5183,00086994002,480
2021-07-07 00:26:00,2021-07-07 00:56:00,SGR 0755-2933,00095967013,1800
2021-07-07 00:56:00,2021-07-07 01:06:00,PROXIMA CEN,00096044014,600
2021-07-07 01:06:00,2021-07-07 01:17:00,ZTF21abioeyq,00014408010,660
2021-07-07 01:17:00,2021-07-07 01:34:00,Mrk501,00011184135,1020
2021-07-07 01:34:00,2021-07-07 01:59:00,M31N 1923-12c,00014412005,1500
2021-07-07 01:59:00,2021-07-07 02:13:00,OGP_5247,03100787004,840


That's a lot of targets! Note that like Swift_ObsQuery it's showing you all the individual snapshots during the day. If you want to group these into observations, you can do that. Remember the `observations` method returns a dictionary, with obsid as the key, and the total observation details as the value. You can display it as a table in Jupyter Notebooks simply:

In [7]:
dayplan.observations

Begin,End,Name,Obs Number,Exposure (s)
2021-07-06 23:40:00,2021-07-07 00:00:00,SGR 1830-0645,00013840016,1200
2021-07-07 00:00:00,2021-07-07 01:59:00,M31N 1923-12c,00014412005,1980
2021-07-07 00:08:00,2021-07-07 17:50:00,SC1825,00013984048,2940
2021-07-07 00:18:00,2021-07-07 00:26:00,OGP_5183,00086994002,480
2021-07-07 00:26:00,2021-07-07 23:03:00,SGR 0755-2933,00095967013,2400
2021-07-07 00:56:00,2021-07-07 21:29:00,PROXIMA CEN,00096044014,3420
2021-07-07 01:06:00,2021-07-07 09:15:00,ZTF21abioeyq,00014408010,2640
2021-07-07 01:17:00,2021-07-07 01:34:00,Mrk501,00011184135,1020
2021-07-07 01:59:00,2021-07-07 02:13:00,OGP_5247,03100787004,840
2021-07-07 02:13:00,2021-07-07 09:05:00,GRB 210702A,01058804007,4980
